In [1]:
import sys
sys.path.append('/Users/user/meepc')
import numpy as np
import pandas as pd
from models import Hankel,Corrhankel,RobustPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
hankel = Hankel()
corrhankel = Corrhankel() 
lag = 360
stride = 0.5


In [3]:
def flatten_pca(list_of_lists):
  flattened_arrays = []
  for sublist in list_of_lists:
    for array in sublist:
      for element in array:
        flattened_arrays.append(element)
      # flattened_arrays.append(array.flatten())
  # print(len(flattened_arrays))
  flattened_arrays=np.unique(flattened_arrays)
  return flattened_arrays


def flatten_meepc(list_of_lists):
  flattened_arrays = []
  for sublist in list_of_lists:
    for array in sublist:
      for element in array:
        flattened_arrays.append(element)
  flattened_arrays=np.unique(np.concatenate(flattened_arrays))
  return flattened_arrays

In [4]:
train_normal = pd.read_csv('~/data/te/normal_training.csv')
train_attack = pd.read_csv('~/data/te/attack_training.csv')
train_normal = pd.concat((train_normal,train_attack),axis=0,ignore_index=True)
y_truth = train_normal.loc[:,'LABEL']

In [5]:
sensors = [col for col in train_normal.columns if col not in ['faultNumber', 'simulationRun', 'sample','LABEL']]

In [6]:
#attack indexes in original training data
temp = hankel.fit(np.array(y_truth) ,lag,stride)
attack_idx = np.any(temp>0,axis=0).astype(int)
attack_idx = np.where(attack_idx>0)[0]
attack_idx

array([1387, 1388, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397,
       1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408,
       1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1419,
       1420, 1421, 1422, 1423, 1424, 1425, 1426, 1427, 1428, 1429, 1430,
       1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441,
       1442], dtype=int64)

In [7]:
len(attack_idx)/len(temp)

0.15555555555555556

In [8]:
scaler = StandardScaler()
X_normal = pd.DataFrame(index=train_normal.index, columns=sensors, data=scaler.fit_transform(train_normal[sensors]))
X_attack = pd.DataFrame(data=scaler.fit_transform(train_attack.loc[:,sensors]),index=train_attack.index,columns=sensors).reset_index().drop(columns=['index'])

In [9]:
corr_normal,nolag_normal = corrhankel.fit(X_normal.to_numpy(),lag,stride)

In [10]:
corr_attack,nolag_attack = corrhankel.fit(X_attack.to_numpy(),lag,stride)

In [11]:
df_test1 = pd.read_csv('~/data/te/normal_testing.csv')
df_test2 = pd.read_csv('~/data/te/attack_testing.csv')
df_test=pd.concat((df_test1,df_test2),axis=0)

# Epasad with 1 cluster (Normal data)

In [12]:
X_test = pd.DataFrame(index=df_test.index, columns=sensors, data=scaler.fit_transform(df_test[sensors]))
Y_test = df_test.loc[:,'LABEL']

In [13]:
labels = hankel.fit(np.array(Y_test),lag,stride)
y_actual = np.any(labels>0,axis=0).astype(int)

In [14]:
sensor_models = []
sensor_predicted = []
accuracy = []
precision = []
recall = []
fscore = []
cluster_indexes=[]
pca_indexes=[]
meepc_indexes=[]
pca_counts=0
meepc_counts=0

for i,sens in enumerate(sensors):
    print("sensor -> ", sens)
    train_normal = X_normal.loc[:,sens].values
    model = RobustPipeline()
    model.fit(train_normal,train_attack,lag,stride,optimal_k=1,kscore_init='inertia',tune=False,alpha=0.15,y_truth=y_truth)
    test = X_test.loc[:,sens].values
    y_predicted = model.predict(test)
    sensor_predicted.append(y_predicted)
    accuracy.append(accuracy_score(y_actual,y_predicted))
    precision.append(precision_score(y_actual,y_predicted))
    recall.append(recall_score(y_actual,y_predicted))
    fscore.append(f1_score(y_actual,y_predicted))

    #clustering
    cluster_indexes.append(model.cluster_idx)
    #pca
    pca_indexes.append(model.pca_idx)
    
    #meepc
    # meepc_counts=max(meepc_counts,model.counts_meepc)
    meepc_indexes.append(model.total_meepc_attacks)
    
    

sensor ->  xmeas_1
------[CLUSTER] attack points found is 34 
------[PCA] attack points found is 56 
1443 [1260 1438 1415  440 1331 1406 1404 1435 1436 1420 1410 1419 1405 1421
 1408 1437 1422 1390 1411 1391 1409 1389]
1421 [ 973 1417  300  339   32 1291  155 1393 1211  860  128  565  945 1385
 1398  564 1396 1405  829 1395 1085   31]
1399 [ 220 1316  689  242   45  296  133  373 1114 1193 1370 1120 1158 1396
 1381  128 1383  778 1236 1384  350  931]
1377 [ 545 1046  738  899  167  349  978 1360  680   16  239  940  609  354
 1153  162  574  169 1012   25 1240 1354]
1355 [ 229  755  843  596   16  929    6 1201  329  842  444  926  953 1280
  122  981 1334 1353  118  550 1333 1240]
1333 [ 932  900  107  926  533   93  592  782  774 1224  637  863  442  267
 1016  582  221  211  508 1240 1055  298]
1311 [ 102  799 1185 1098    7   63  704  467  573  393  920 1048  375 1128
  797  511 1295  774  371  252  413  445]
1289 [ 953  188  107  360  130  429   18 1244  201 1075  965  570  352  8

In [15]:
attack_idx 

array([1387, 1388, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397,
       1398, 1399, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408,
       1409, 1410, 1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1419,
       1420, 1421, 1422, 1423, 1424, 1425, 1426, 1427, 1428, 1429, 1430,
       1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441,
       1442], dtype=int64)

In [16]:
cluster_inactive_idx = np.unique(np.concatenate(cluster_indexes))
commonidxs=np.intersect1d(attack_idx,cluster_inactive_idx)
percentage = len(commonidxs)/len(attack_idx)*100
print(percentage)
print(commonidxs)



100.0
[1387 1388 1389 1390 1391 1392 1393 1394 1395 1396 1397 1398 1399 1400
 1401 1402 1403 1404 1405 1406 1407 1408 1409 1410 1411 1412 1413 1414
 1415 1416 1417 1418 1419 1420 1421 1422 1423 1424 1425 1426 1427 1428
 1429 1430 1431 1432 1433 1434 1435 1436 1437 1438 1439 1440 1441 1442]


In [17]:
pca_inactive_idx = np.unique(np.concatenate(pca_indexes))
commonidxs=np.intersect1d(attack_idx,pca_inactive_idx)
percentage = len(commonidxs)/len(attack_idx)*100
print(percentage) 
print(commonidxs)

100.0
[1387 1388 1389 1390 1391 1392 1393 1394 1395 1396 1397 1398 1399 1400
 1401 1402 1403 1404 1405 1406 1407 1408 1409 1410 1411 1412 1413 1414
 1415 1416 1417 1418 1419 1420 1421 1422 1423 1424 1425 1426 1427 1428
 1429 1430 1431 1432 1433 1434 1435 1436 1437 1438 1439 1440 1441 1442]


In [18]:
# percentage = meepc_counts/len(attack_idx)*100
# print(percentage) 
meepc_inactive_idx = np.unique(np.concatenate(meepc_indexes))
commonidxs=np.intersect1d(attack_idx,meepc_inactive_idx)
percentage = len(commonidxs)/len(attack_idx)*100
print(percentage) 
print(commonidxs)

100.0
[1387 1388 1389 1390 1391 1392 1393 1394 1395 1396 1397 1398 1399 1400
 1401 1402 1403 1404 1405 1406 1407 1408 1409 1410 1411 1412 1413 1414
 1415 1416 1417 1418 1419 1420 1421 1422 1423 1424 1425 1426 1427 1428
 1429 1430 1431 1432 1433 1434 1435 1436 1437 1438 1439 1440 1441 1442]


In [19]:
sensor_predicted = np.asarray(sensor_predicted)
y_predicted = np.any(sensor_predicted,axis=0).astype(int)
y_predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
print("Accuracy ",accuracy_score(y_actual,y_predicted))
print("Precision ",precision_score(y_actual,y_predicted))
print("Recall ",recall_score(y_actual,y_predicted))
print("F1-score ",f1_score(y_actual,y_predicted))

Accuracy  0.19090909090909092
Precision  1.0
Recall  0.16037735849056603
F1-score  0.2764227642276422


In [21]:
print("Accuracy -  Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(accuracy).mean(), np.median(np.asarray(accuracy)),np.asarray(accuracy).min(), np.asarray(accuracy).max()))
print("precision - Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(precision).mean(), np.median(np.asarray(precision)),np.asarray(precision).min(), np.asarray(precision).max()))
print("recall -    Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(recall).mean(), np.median(np.asarray(recall)),np.asarray(recall).min(), np.asarray(recall).max()))
print("f1 -        Mean: {} Median : {} Min : {} Max : {}".format(np.asarray(fscore).mean(), np.median(np.asarray(fscore)),np.asarray(fscore).min(), np.asarray(fscore).max()))

Accuracy -  Mean: 0.041783216783216774 Median : 0.03636363636363636 Min : 0.03636363636363636 Max : 0.08181818181818182
precision - Mean: 0.23076923076923078 Median : 0.0 Min : 0.0 Max : 1.0
recall -    Mean: 0.0056240928882438325 Median : 0.0 Min : 0.0 Max : 0.04716981132075472
f1 -        Mean: 0.010920540917111267 Median : 0.0 Min : 0.0 Max : 0.09009009009009009


# Multiple clusters + No threshold tuning

In [22]:
cluster_fscore = []
for k in range(2,5):
    print('-------------------------------------',k,'-------------------------------------')
    
    sensor_predicted = []
    accuracy = []
    precision = []
    recall = []
    fscore = []
    cluster_indexes=[]
    pca_indexes=[]
    meepc_indexes=[]
    pca_counts=0
    meepc_counts=0
    for i,sens in enumerate(sensors):
        print(sens)
        train_normal = X_normal.loc[:,sens].values
        train_attack = X_attack.loc[:,sens].values
        model = RobustPipeline()
        model.fit(train_normal,train_attack,lag,stride,optimal_k=k,tune=False,kscore_init='inertia',alpha=0.15,y_truth=y_truth)
        test = X_test.loc[:,sens].values
        y_predicted = model.predict(test)
        sensor_predicted.append(y_predicted)
        accuracy.append(accuracy_score(y_actual,y_predicted)) 
        precision.append(precision_score(y_actual,y_predicted))
        recall.append(recall_score(y_actual,y_predicted))
        fscore.append(f1_score(y_actual,y_predicted))

        cluster_indexes.append(model.cluster_idx)
        pca_indexes.append(model.pca_idx)
        pca_counts=max(pca_counts,model.total_pca_attacks)
        # meepc_counts=max(meepc_counts,model.counts_meepc)
        meepc_indexes.append(model.total_meepc_attacks)
    
            
    sensor_predicted = np.asarray(sensor_predicted)
    y_predicted = np.any(sensor_predicted,axis=0).astype(int)
    print('------Number of Clusters: ',k,'-----------') 
    print("Accuracy ",accuracy_score(y_actual,y_predicted))
    print("Precision ",precision_score(y_actual,y_predicted))
    print("Recall ",recall_score(y_actual,y_predicted))
    print("F1-score ",f1_score(y_actual,y_predicted),'\n')

    #cluster
    cluster_inactive_idx = np.unique(np.concatenate(cluster_indexes))
    commonidxs=np.intersect1d(attack_idx,cluster_inactive_idx)
    percentage = len(commonidxs)/len(attack_idx)*100
    print("Cluster - attack points detected {} %".format(percentage))
    #PCA
    
    # percentage = pca_counts/len(attack_idx)*100
    # print("PCA - attack points detected {} %".format(percentage))
    pca_inactive_idx = flatten_pca(pca_indexes)
    commonidxs=np.intersect1d(attack_idx,pca_inactive_idx)
    percentage = len(commonidxs)/len(attack_idx)*100
    print("PCA - attack points detected {} %".format(percentage))


    #Meepc
    # percentage = meepc_counts/len(attack_idx)*100
    # print("Meepc - attack points detected {} %".format(percentage))
    meepc_inactive_idx = flatten_meepc(meepc_indexes)
    commonidxs=np.intersect1d(attack_idx,meepc_inactive_idx)
    percentage = len(commonidxs)/len(attack_idx)*100
    print("MEEPC - attack points detected {} %".format(percentage))
    
    cluster_fscore.append(f1_score(y_actual,y_predicted))

------------------------------------- 2 -------------------------------------
xmeas_1
------[CLUSTER] attack points found is 32 
------[PCA] attack points found is 35 
839 [812 814 817 559 821  18 822 823 423 811 277 322]
827 [805  64 821 652 334 824 806 802 823  26 110 560]
815 [812 802 391  39 736 367 370 589  85 231 316 138]
803 [785 497  13 192 697 801   9 696 702 786 793 321]
791 [157 780 395 231 249 360 782 247  81 671 455  70]
779 [355 659 233 336 624 678 280 592 646 329 230  65]
767 [469 723 162 435 697 189 130 634 755 334 754 757]
755 [747 122 677 219 430 197 676 745 173 117  67 471]
743 [102 705 443   3  34 736 270 369 624 597 202 121]
731 [123 419 230  57 574 111 153  87 624 566 326 353]
------[Meepc] attack points found is 27 
------[PCA] attack points found is 21 
604 [302  61 584 603 239 140 559 339 386]
595 [588 496 587 590 578  90 523 589 403]
586 [423 581  39 585 166 296 573 146 431]
577 [ 48  72 566 573 101 275 129 401 452]
568 [406 506 138  89 379 414 323 238 564]
55

In [23]:
cluster_fscore


[0.4857142857142857, 0.4964539007092198, 0.4525547445255474]